In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

## Configure notebook and get raw data

In [ ]:
# notebook config
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
sns.set_context("notebook")

# get raw data
data_path = Path("../data/MACH_data/data.cleaned.csv")
codebook_path = Path("../data/MACH_data/codebook.txt")
df = pd.read_csv(data_path)

# show raw data
print(f"Shape: {df.shape}")
display(df.head())

## Choose features
Sticking to just the actual question response for now.

In [ ]:

question_responses = ["Q1A", "Q2A", "Q3A", "Q4A", "Q5A", "Q6A", "Q7A", "Q8A", "Q9A", "Q10A", 
                      "Q11A", "Q12A", "Q13A", "Q14A", "Q15A", "Q16A", "Q17A", "Q18A", "Q19A", "Q20A"]
X = df[question_responses].copy()
print(f"Using features: {question_responses}  |  Shape: {X.shape}")
display(X.head())

## Data preprocessing
Scalaing this data is likely not necessary since the data is usually between 1 and 5, with some NA values.

In [ ]:
X_clean = X.dropna().copy()
X_clean[question_responses] = X_clean[question_responses].astype(int)
Xs = pd.DataFrame(X_clean, columns=question_responses, index=X_clean.index)
# use only 5000 random datapoints
Xs_sample = Xs.sample(n=5000, random_state=42)
print(f"After preprocessing shape: {Xs_sample.shape}")
display(Xs_sample.head())
display(Xs_sample.describe().T.round(3))

## Compute pairwise distances and distance matrix manually
This may not be necessary for the linkage step.

In [ ]:
euclidean = pdist(Xs_sample, metric="euclidean")
euclidean_condensed = squareform(euclidean)
manhattan = pdist(Xs_sample, metric="cityblock")
manhattan_condensed = squareform(manhattan)

print(f"Condensed Euclidean shape: {euclidean_condensed.shape}")
print(f"Euclidean 5x5 Slice:\n{euclidean_condensed[0:5, 0:5]}")
print(f"Manhattan Euclidean shape: {manhattan_condensed.shape}")
print(f"Mahattan 5x5 Slice:\n{manhattan_condensed[0:5, 0:5]}")

## Compute linkage (Euclidean vs. Manhattan vs. no distance matrix)
Ward linkage is a special case where pre-computed pairwise distances is invalid since Ward linkage requires raw observations.

In [ ]:
Z_single_euclidean = linkage(euclidean_condensed, method="single")
Z_complete_euclidean = linkage(euclidean_condensed, method="complete")
Z_average_euclidean = linkage(euclidean_condensed, method="average")

In [ ]:
Z_single_manhattan = linkage(manhattan_condensed, method="single")
Z_complete_manhattan = linkage(manhattan_condensed, method="complete")
Z_average_manhattan = linkage(manhattan_condensed, method="average")

In [ ]:
Z_single_default = linkage(Xs_sample, method="single")
Z_complete_default = linkage(Xs_sample, method="complete")
Z_average_default = linkage(Xs_sample, method="average")
Z_ward = linkage(Xs_sample, method="ward")

## Plot dendrograms

In [ ]:
# auto params per tree
p_show = 20
ct_single = 0.7 * Z_single_default[:, 2].max()
ct_complete = 0.7 * Z_complete_default[:, 2].max()
ct_average = 0.7 * Z_average_default[:, 2].max()
ct_ward = 0.7 * Z_ward[:, 2].max()

fig, ax = plt.subplots(1, 4, figsize=(12, 6))

# single linkage
dendrogram(
    Z_single_default,
    truncate_mode="lastp",
    p=min(p_show, Z_single_default.shape[0] + 1),
    color_threshold=ct_single,
    no_labels=True,
    count_sort="ascending",
    distance_sort="descending",
    ax=ax[0]
)
ax[0].set_title("Single linkage dendrogram")
ax[0].set_xlabel("Merged leaves")
ax[0].set_ylabel("Distance")

# complete linkage
dendrogram(
    Z_complete_default,
    truncate_mode="lastp",
    p=min(p_show, Z_complete_default.shape[0] + 1),
    color_threshold=ct_complete,
    no_labels=True,
    count_sort="ascending",
    distance_sort="descending",
    ax=ax[1]
)
ax[1].set_title("Complete linkage dendrogram")
ax[1].set_xlabel("Merged leaves")
ax[1].set_ylabel("Distance")

# average linkage
dendrogram(
    Z_average_default,
    truncate_mode="lastp",
    p=min(p_show, Z_average_default.shape[0] + 1),
    color_threshold=ct_average,
    no_labels=True,
    count_sort="ascending",
    distance_sort="descending",
    ax=ax[2]
)
ax[2].set_title("Average linkage dendrogram")
ax[2].set_xlabel("Merged leaves")
ax[2].set_ylabel("Distance")

# ward linkage
dendrogram(
    Z_ward,
    truncate_mode="lastp",
    p=min(p_show, Z_ward.shape[0] + 1),
    color_threshold=ct_ward, # ward has much larger heights, so use its own max
    no_labels=True,
    count_sort="ascending",
    distance_sort="descending",
    ax=ax[3]
)
ax[3].set_title("Ward linkage dendrogram")
ax[3].set_xlabel("Merged leaves")
ax[3].set_ylabel("Distance")

plt.suptitle("No manual distance calculations")
plt.tight_layout()
plt.show()

In [ ]:
# auto params per tree
p_show = 20
ct_single = 0.7 * Z_single_euclidean[:, 2].max()
ct_complete = 0.7 * Z_complete_euclidean[:, 2].max()
ct_average = 0.7 * Z_average_euclidean[:, 2].max()
ct_ward = 0.7 * Z_ward[:, 2].max()

fig, ax = plt.subplots(1, 4, figsize=(12, 6))

# single linkage
dendrogram(
    Z_single_euclidean, 
    truncate_mode="lastp",
    p=min(p_show, Z_single_euclidean.shape[0] + 1),
    color_threshold=ct_single,
    no_labels=True,
    count_sort="ascending",
    distance_sort="descending",
    ax=ax[0]
)
ax[0].set_title("Single linkage dendrogram")
ax[0].set_xlabel("Merged leaves")
ax[0].set_ylabel("Distance")

# complete linkage
dendrogram(
    Z_complete_euclidean,
    truncate_mode="lastp",
    p=min(p_show, Z_complete_euclidean.shape[0] + 1),
    color_threshold=ct_complete,
    no_labels=True,
    count_sort="ascending",
    distance_sort="descending",
    ax=ax[1]
)
ax[1].set_title("Complete linkage dendrogram")
ax[1].set_xlabel("Merged leaves")
ax[1].set_ylabel("Distance")

# average linkage
dendrogram(
    Z_average_euclidean,
    truncate_mode="lastp",
    p=min(p_show, Z_average_euclidean.shape[0] + 1),
    color_threshold=ct_average,
    no_labels=True,
    count_sort="ascending",
    distance_sort="descending",
    ax=ax[2]
)
ax[2].set_title("Average linkage dendrogram")
ax[2].set_xlabel("Merged leaves")
ax[2].set_ylabel("Distance")

# ward linkage
dendrogram(
    Z_ward,
    truncate_mode="lastp",
    p=min(p_show, Z_ward.shape[0] + 1),
    color_threshold=ct_ward, # ward has much larger heights, so use its own max
    no_labels=True,
    count_sort="ascending",
    distance_sort="descending",
    ax=ax[3]
)
ax[3].set_title("Ward linkage dendrogram")
ax[3].set_xlabel("Merged leaves")
ax[3].set_ylabel("Distance")

plt.suptitle("Manual Euclidean distance calculations")
plt.tight_layout()
plt.show()

In [ ]:
# auto params per tree
p_show = 20
ct_single = 0.7 * Z_single_manhattan[:, 2].max()
ct_complete = 0.7 * Z_complete_manhattan[:, 2].max()
ct_average = 0.7 * Z_average_manhattan[:, 2].max()
ct_ward = 0.7 * Z_ward[:, 2].max()

fig, ax = plt.subplots(1, 4, figsize=(12, 6))

# single linkage
dendrogram(
    Z_single_manhattan,
    truncate_mode="lastp",
    p=min(p_show, Z_single_manhattan.shape[0] + 1),
    color_threshold=ct_single,
    no_labels=True,
    count_sort="ascending",
    distance_sort="descending",
    ax=ax[0]
)
ax[0].set_title("Single linkage dendrogram")
ax[0].set_xlabel("Merged leaves")
ax[0].set_ylabel("Distance")

# complete linkage
dendrogram(
    Z_complete_manhattan,
    truncate_mode="lastp",
    p=min(p_show, Z_complete_manhattan.shape[0] + 1),
    color_threshold=ct_complete,
    no_labels=True,
    count_sort="ascending",
    distance_sort="descending",
    ax=ax[1]
)
ax[1].set_title("Complete linkage dendrogram")
ax[1].set_xlabel("Merged leaves")
ax[1].set_ylabel("Distance")

# average linkage
dendrogram(
    Z_average_manhattan,
    truncate_mode="lastp",
    p=min(p_show, Z_average_manhattan.shape[0] + 1),
    color_threshold=ct_average,
    no_labels=True,
    count_sort="ascending",
    distance_sort="descending",
    ax=ax[2]
)
ax[2].set_title("Average linkage dendrogram")
ax[2].set_xlabel("Merged leaves")
ax[2].set_ylabel("Distance")

# ward linkage
dendrogram(
    Z_ward,
    truncate_mode="lastp",
    p=min(p_show, Z_ward.shape[0] + 1),
    color_threshold=ct_ward, # ward has much larger heights, so use its own max
    no_labels=True,
    count_sort="ascending",
    distance_sort="descending",
    ax=ax[3]
)
ax[3].set_title("Ward linkage dendrogram")
ax[3].set_xlabel("Merged leaves")
ax[3].set_ylabel("Distance")

plt.suptitle("Manual Manhattan distance calculations")
plt.tight_layout()
plt.show()

## Interpretation of dendrograms
1. **Short merge height** --> particpants answered very similarly across 20 questions.
2. **Tall merge height** --> participants answered very differently across 20 questions.
3. **Single Linkage**:
    - Tall chain without clear, wide separations.
    - No clean clusters because single linkage clusters based on closest pairs; everyone linked gradually with incremental differences.
    - *Responses are distributed smoothly, no strong or isolated subgroups.*
4. **Complete Linkage**:
    - Merges based on farthest pairs.
    - Tall last merge and relatively balanced sub-branches suggest a few well-separated groups at the top.
    - *There may be a handful of distinct response profiles, maybe high vs low MACH-IV scores, with smaller inter-group variation.*
5. **Average Linkage**:
    - Averages all pairwise distances between clusters.
    - Similar to complete linkage but smoother, indicating moderately distinct and overlapping groups.
    - *Multiple gradations of Machiavellian tendencies rather than sharply distinct clusters.*
6. **Ward Linkage**:
    - Minimizes inter-cluster variance.
    - Larger distance due to sqaured Euclidean distance.
    - Not appropriate to pre-compute distance matrices since it essentially does that internally.
    - Clear branching: 2-3 big clusters (depending on what you consider large) before a massive final merge.
    - *Likely the most informative, suggests 2-3 main subgroups, perhaps low-mach, medium-mach, high-mach.*
## Main takeaways
- The structure is more graded, which suggests Machiavellianism is a spectrum rather than its own distinct personality type (aligns with accepted understanding).
- **Ward linkage** offers the most interpretable hierarchy for psychological traits; could be cut into 2-3 clusters.
- Manhattan distance could *slightly* improve interpretability.

### Sticking with Ward linkage w/ 2, 3, or 4 clusters

In [ ]:
ward_labels = fcluster(Z_ward, 3, criterion="maxclust") # criterion="distance"
df = Xs_sample.copy()
df["Cluster"] = ward_labels
print(f"Sample per cluster: {df["Cluster"].value_counts()}")

In [ ]:
for k in [2, 3, 4]:
    labels = fcluster(Z_ward, k, criterion="maxclust")
    score = silhouette_score(Xs_sample, labels)
    print(f"Silhouette score for {k} clusters: {score:.3f}")

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(Xs_sample)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, k in enumerate([2, 3, 4]):
    labels = fcluster(Z_ward, k, criterion="maxclust")
    score = silhouette_score(Xs_sample, labels)
    axes[i].scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap="tab10", s=15)
    axes[i].set_title(f"PCA Projection ({k} clusters)\nSilhouette: {score:.3f}")
    axes[i].set_xlabel("PCA 1")
    axes[i].set_ylabel("PCA 2")
plt.tight_layout()
plt.show()

## Interpretation of PCA projections
1. **2 clusters** --> simple high/low Machiavellianism clusters.
2. **3 clusters** --> low-Mach (disagreeing with manipulative tendencies), mid-Mach (moderates or situational), and high-Mach (agreeing with manipulative or pragmatic tendencies).
3. **4 clusters** --> likely overfitted, may be useful for k=4 or more if mean response per cluster is further disected.

### Analyze cluster mean responses

In [ ]:
cluster_means = df.groupby("Cluster").mean()
display(cluster_means)

plt.figure(figsize=(10, 6))
sns.heatmap(cluster_means, annot=True, cmap="coolwarm", cbar_kws={"label": "Mean Response"})
plt.title("Mean MACH-IV Question Responses per Cluster")
plt.xlabel("Question")
plt.ylabel("Cluster")
plt.show()

In [ ]:
cluster_modes = df.groupby("Cluster")[question_responses] \
                  .agg(lambda x: x.mode().iloc[0]) # first mode
display(cluster_modes)

plt.figure(figsize=(10, 6))
sns.heatmap(cluster_modes, annot=True, cmap="coolwarm", cbar_kws={"label": "Median Response"})
plt.title("Mode MACH-IV Question Responses per Cluster")
plt.xlabel("Question")
plt.ylabel("Cluster")
plt.show()

### Psychological Summary
**Cluster 1:** Low-Mach - ethical idealists<br>
**Cluster 2:** High-Mach - strategic manipulators<br>
**Cluster 3:** Mid-Mach (or Situational-Mach) - contextual pragmatists

**Main Takeaway:** Consistent with theoretical Machiavellian spectrum proposed in the 1970s.

### Most Machiavellian Response Per Question
Q1A:  5<br>
Q2A:  5<br>
Q3A:  1<br>
Q4A:  1<br>
Q5A:  5<br>
Q6A:  1<br>
Q7A:  1<br>
Q8A:  5<br>
Q9A:  1<br>
Q10A: 1<br>
Q11A: 1<br>
Q12A: 5<br>
Q13A: 5<br>
Q14A: 1<br>
Q15A: 5<br>
Q16A: 1<br>
Q17A: 1<br>
Q18A: 5<br>
Q19A: 5<br>
Q20A: 5